# GPR-FedSense 实验：Baseline vs Full

本 Notebook 运行两个实验：
1. **Baseline**: 纯 FedDWA（不使用 FedVLS 和 FedDecorr）
2. **GPR-FedSense**: FedDWA + FedVLS + FedDecorr

实验设置：
- 数据集：GPR Custom
- 模型：MobileViT-S
- Non-IID 类型：Type-8 (病态分割)
- 客户端参与率：30% (困难模式)
- 全局轮次：50

In [ ]:
# ============================================
# Cell 1: 环境准备
# ============================================
import os
import shutil

# 1. 复制代码到工作目录
if not os.path.exists('/kaggle/working/FedDWA'):
    shutil.copytree('/kaggle/input/d/kkyylove/feddwa-codev5-0/FedDWA-main', '/kaggle/working/FedDWA')
    print("✓ 代码已复制到工作目录")

# 2. 切换工作目录
os.chdir('/kaggle/working/FedDWA')
print(f"✓ 当前工作目录: {os.getcwd()}")

# 3. 安装依赖
!pip install -q -r requirements.txt
print("✓ 依赖安装完成")

In [ ]:
# ============================================
# Cell 2: 修复依赖冲突
# ============================================
print("正在清理冲突的包...")
!pip uninstall -y numpy scipy opencv-python opencv-contrib-python opencv-python-headless 2>/dev/null || true

print("\n安装兼容版本的核心库...")
!pip install --quiet \
    numpy==1.26.4 \
    scipy==1.11.4 \
    matplotlib==3.7.5 \
    opencv-python==4.8.1.78 \
    albumentations \
    tensorboardX \
    thop \
    seaborn \
    scikit-learn

print("✓ 依赖冲突已解决")

In [ ]:
# ============================================
# Cell 3: 实验 1 - Baseline (不使用 FedSense)
# ============================================
print("\n" + "="*60)
print("实验 1: Baseline (FedDWA Only)")
print("="*60 + "\n")

!python main.py \
    --dataset gpr_custom \
    --data_dir /kaggle/input/data-oll/data \
    --model mobilevit_s \
    --alg feddwa \
    --client_num 5 \
    --client_frac 0.3 \
    --global_rounds 50 \
    --E 3 \
    --B 16 \
    --lr 0.001 \
    --non_iidtype 8 \
    --alpha_dir 0.1 \
    --device cuda \
    --seed 42

# 保存基线结果
print("\n保存 Baseline 结果...")
!mkdir -p /kaggle/working/results
!cp -r logs_feddwa /kaggle/working/results/logs_baseline
!cp feddwa_analysis_result.png /kaggle/working/results/baseline_result.png 2>/dev/null || true
!cp *.csv /kaggle/working/results/baseline_results.csv 2>/dev/null || true
!cp -r confusion_matrix_* /kaggle/working/results/baseline_confusion/ 2>/dev/null || true
!cp -r client_confusion_matrices /kaggle/working/results/baseline_client_confusion/ 2>/dev/null || true
print("✓ Baseline 结果已保存")

In [ ]:
# ============================================
# Cell 4: 实验 2 - GPR-FedSense (完整版)
# ============================================
print("\n" + "="*60)
print("实验 2: GPR-FedSense (FedVLS + FedDecorr)")
print("="*60 + "\n")

!python main.py \
    --dataset gpr_custom \
    --data_dir /kaggle/input/data-oll/data \
    --model mobilevit_s \
    --alg feddwa \
    --client_num 5 \
    --client_frac 0.3 \
    --global_rounds 50 \
    --E 3 \
    --B 16 \
    --lr 0.001 \
    --non_iidtype 8 \
    --alpha_dir 0.1 \
    --device cuda \
    --seed 42 \
    --use_fedvls --fedvls_alpha 1.0 \
    --use_feddecorr --feddecorr_beta 0.1

# 保存 FedSense 结果
print("\n保存 GPR-FedSense 结果...")
!cp -r logs_feddwa /kaggle/working/results/logs_fedsense
!cp feddwa_analysis_result.png /kaggle/working/results/fedsense_result.png 2>/dev/null || true
!cp *.csv /kaggle/working/results/fedsense_results.csv 2>/dev/null || true
!cp -r confusion_matrix_* /kaggle/working/results/fedsense_confusion/ 2>/dev/null || true
!cp -r client_confusion_matrices /kaggle/working/results/fedsense_client_confusion/ 2>/dev/null || true
print("✓ GPR-FedSense 结果已保存")

In [ ]:
# ============================================
# Cell 5: 结果对比可视化（使用新的 plot_utils）
# ============================================
import sys
sys.path.append('/kaggle/working/FedDWA')

from utils.plot_utils import plot_comparison
from IPython.display import Image, display

print("\n" + "="*60)
print("生成对比图...")
print("="*60 + "\n")

# 使用新的 plot_comparison 函数
plot_comparison(
    log_dirs=[
        '/kaggle/working/results/logs_baseline',
        '/kaggle/working/results/logs_fedsense'
    ],
    labels=['Baseline (FedDWA)', 'GPR-FedSense'],
    save_path='/kaggle/working/results/comparison_result.png',
    colors=['#3498db', '#e74c3c'],
    title='GPR-FedSense Performance Comparison'
)

print("\n✓ 对比图已生成")

In [ ]:
# ============================================
# Cell 6: 显示所有结果
# ============================================
from IPython.display import Image, display
import os

print("\n" + "="*60)
print("实验结果可视化")
print("="*60 + "\n")

# 1. 对比图（最重要）
print("\n📊 实验对比图:")
if os.path.exists('/kaggle/working/results/comparison_result.png'):
    display(Image(filename='/kaggle/working/results/comparison_result.png'))
else:
    print("❌ 未找到对比图")

# 2. Baseline 训练曲线
print("\n📈 Baseline 训练曲线:")
if os.path.exists('/kaggle/working/results/baseline_result.png'):
    display(Image(filename='/kaggle/working/results/baseline_result.png'))
else:
    print("❌ 未找到 Baseline 训练曲线")

# 3. GPR-FedSense 训练曲线
print("\n📈 GPR-FedSense 训练曲线:")
if os.path.exists('/kaggle/working/results/fedsense_result.png'):
    display(Image(filename='/kaggle/working/results/fedsense_result.png'))
else:
    print("❌ 未找到 GPR-FedSense 训练曲线")

# 4. 全局混淆矩阵对比
print("\n🔍 全局混淆矩阵对比:")
print("\nBaseline:")
baseline_cm = '/kaggle/working/results/baseline_confusion/confusion_matrix_global.png'
if os.path.exists(baseline_cm):
    display(Image(filename=baseline_cm))

print("\nGPR-FedSense:")
fedsense_cm = '/kaggle/working/results/fedsense_confusion/confusion_matrix_global.png'
if os.path.exists(fedsense_cm):
    display(Image(filename=fedsense_cm))

print("\n✓ 所有结果已显示")

In [ ]:
# ============================================
# Cell 7: 下载结果文件（可选）
# ============================================
print("\n" + "="*60)
print("结果文件列表")
print("="*60 + "\n")

!ls -lh /kaggle/working/results/

print("\n提示: 您可以在 Kaggle 的 Output 标签页下载以下文件:")
print("  - comparison_result.png (对比图)")
print("  - baseline_results.csv (基线数据)")
print("  - fedsense_results.csv (FedSense 数据)")
print("  - logs_baseline/ (基线日志)")
print("  - logs_fedsense/ (FedSense 日志)")